In [1]:
from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from aap_core.agent import AgentMessage, BaseAgent
from aap_core.chain import BaseLLMChain
from aap_core.orchestration import DebateAgent
from aap_langchain.chain import ChatCausalMultiTurnsChain
from langchain_ollama import ChatOllama

In [2]:
class Agent(BaseAgent):
    chain: BaseLLMChain

    def execute(self, message: AgentMessage, **kwargs) -> AgentMessage:
        self.state = "running"
        message = self.chain.invoke(message, **kwargs)
        message.execution_result = "success"
        message.origin = self.card.name
        self.state = "idle"
        return message

In [3]:
# System prompts generated by Gemini
system_prompt_1 = """You are a Arboreal Protector who is an elite environmental agent dedicated to combating the complex challenges threatening the world's forests and vital ecosystems. Specializing in issues from illegal logging and rampant deforestation to the impacts of climate change and invasive species, this operative combines deep ecological knowledge with advanced surveillance and strategic planning. They are adept at navigating dense wilderness for fieldwork, utilizing remote sensing technologies to monitor forest health, and coordinating international efforts to secure protected areas. Whether tracking illicit timber operations or developing sustainable land management plans alongside local communities, the Arboreal Protector is the vigilant, resourceful shield ensuring the longevity and biodiversity of our planet's indispensable woodlands.
"""

system_prompt_2 = """You are a Fauna Guardian who is a highly specialized environmental operative focused on the urgent protection of endangered animal species, their critical habitats, and the prevention of biodiversity loss. This agent possesses a unique blend of veterinary medicine, behavioral ecology, and conservation law expertise, enabling them to tackle immediate threats like poaching, habitat fragmentation, and wildlife trafficking. Their work involves conducting clandestine anti-poaching missions, establishing and maintaining safe breeding programs, and using sophisticated tracking technology to monitor at-risk populations. The Fauna Guardian collaborates closely with global scientific bodies and local rangers, acting as the crucial frontline defender committed to ensuring that vulnerable species, from keystone predators to delicate endemic organisms, are safeguarded from the precipice of extinction."""

system_prompt_3 = """You are an industrial Decarbonization Strategist who is an expert in corporate and industrial governance, keenly focused on guiding large-scale enterprises toward achieving rigorous Net Zero goals and fundamentally greening their production processes. This professional excels at integrating sustainability mandates directly into high-level business strategy, supply chain management, and regulatory compliance frameworks. They possess deep knowledge of carbon accounting, renewable energy procurement, and the deployment of circular economy principles to minimize waste and resource consumption. The Strategist’s role is to bridge the gap between financial performance and ecological responsibility, advising boards and executive teams on the necessary technological transitions and operational shifts required to make industrial output truly "Greenery," ensuring profitability aligns with a verifiable, accelerated path toward climate neutrality."""

user_prompt = """{query}
"""

def state_callback(state: str):
    print(f"agent state: {state}")

agent1_skill = AgentSkill(
    id='agent1-skill',
    name="agent1 skill",
    description="self-agent1 skill",
    tags=['agent1']
)
agent1_card = AgentCard(
    name="agent1",
    description="self-agent1 agent",
    skills=[agent1_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
agent2_skill = AgentSkill(
    id='agent2-skill',
    name="agent2 skill",
    description="self-agent2 skill",
    tags=['agent2']
)
agent2_card = AgentCard(
    name="agent2",
    description="self-agent2 agent",
    skills=[agent2_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
agent3_skill = AgentSkill(
    id='agent3-skill',
    name="agent3 skill",
    description="self-agent3 skill",
    tags=['agent3']
)
agent3_card = AgentCard(
    name="agent3",
    description="self-agent3 agent",
    skills=[agent3_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
debate_skill = AgentSkill(
    id='debate-skill',
    name="debate skill",
    description="self-debate skill",
    tags=['debate']
)
debate_card = AgentCard(
    name="debate agent",
    description="self-debate agent",
    skills=[debate_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)

model1 = ChatOllama(model="qwen3:4b-thinking-2507-q4_K_M", base_url="192.168.55.1:11434", temperature=1.3)
model2 = ChatOllama(model="mistral:7b-instruct-v0.3-q4_K_S", base_url="192.168.55.1:11434", temperature=1.3)
model3 = ChatOllama(model="granite3.3:8b", base_url="192.168.55.1:11434", temperature=1.3)

# Include all history
chain1 = ChatCausalMultiTurnsChain(model=model1, system_prompt=system_prompt_1, user_prompt_template=user_prompt, include_history=-1)
chain2 = ChatCausalMultiTurnsChain(model=model2, system_prompt=system_prompt_2, user_prompt_template=user_prompt, include_history=-1)
chain3 = ChatCausalMultiTurnsChain(model=model3, system_prompt=system_prompt_3, user_prompt_template=user_prompt, include_history=-1)

agent1 = Agent(chain=chain1, card=agent1_card, state_change_callback=state_callback)
agent2 = Agent(chain=chain2, card=agent2_card, state_change_callback=state_callback)
agent3 = Agent(chain=chain3, card=agent3_card, state_change_callback=state_callback)

debate_agent = DebateAgent(agents=[agent1, agent2, agent3], card=debate_card, pick_strategy="round_robin", max_turns=7, state_change_callback=state_callback)

In [4]:
debate_agent.max_turns = 7
debate_agent.pick_strategy = "round_robin"

query = "Propose a solution to the problem of global warming."

message = debate_agent.execute(AgentMessage(query=query))

print(f"message len: {len(message.responses)}")
for agent_name, response in message.responses:
    print(f"{agent_name}\n{response}")
    print("-" * 50)
    print()

agent state: debate agent:turn 0: agent agent1 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 0: agent agent1 running/((agent1:running)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 0: agent agent1 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 1: agent agent2 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 1: agent agent2 running/((agent1:idle)-(agent2:running)-(agent3:idle))
agent state: debate agent:turn 1: agent agent2 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 2: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 2: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:running))
agent state: debate agent:turn 2: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 3: agent agent1 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
a

In [5]:
debate_agent.max_turns = 6
debate_agent.pick_strategy = "random"
debate_agent.random_seed = 2025

query = "The ice of the northpole is melting faster than before in recent decades. Poeple even witness the melting of the eternal ice. Propose a solution related to your field to potentially slow down this process."
message = debate_agent.execute(AgentMessage(query=query))

print(f"message len: {len(message.responses)}")
for agent_name, response in message.responses:
    print(f"{agent_name}\n{response}")
    print("-" * 50)
    print()

agent state: debate agent:turn 0: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 0: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:running))
agent state: debate agent:turn 0: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 1: agent agent1 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 1: agent agent1 running/((agent1:running)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 1: agent agent1 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 2: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 2: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:running))
agent state: debate agent:turn 2: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 3: agent agent2 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
a

In [6]:
debate_agent.max_turns = 8
debate_agent.pick_strategy = "simultaneous"

query = """Water is a precious resource for our human kind and our planet. Since industrial revolution, industrial and human activity is increasingly contaminate the water source. Propose a solution to decline this problem in 3 aspects:
- Wasting water
- The quality of water after people have used and sent back to the environment
- Desertification"""
message = debate_agent.execute(AgentMessage(query=query))

print(f"message len: {len(message.responses)}")
for agent_name, response in message.responses:
    print(f"{agent_name}\n{response}")
    print("-" * 50)
    print()

agent state: debate agent:turn 0: all agents running/((agent1:idle)|(agent2:idle)|(agent3:idle))
agent state: debate agent:turn 0: all agents running/((agent1:running)|(agent2:idle)|(agent3:idle))
agent state: debate agent:turn 0: all agents running/((agent1:idle)|(agent2:idle)|(agent3:idle))
agent state: debate agent:turn 0: all agents running/((agent1:idle)|(agent2:running)|(agent3:idle))
agent state: debate agent:turn 0: all agents running/((agent1:idle)|(agent2:idle)|(agent3:idle))
agent state: debate agent:turn 0: all agents running/((agent1:idle)|(agent2:idle)|(agent3:running))
agent state: debate agent:turn 0: all agents running/((agent1:idle)|(agent2:idle)|(agent3:idle))
agent state: debate agent:turn 3: all agents running/((agent1:idle)|(agent2:idle)|(agent3:idle))
agent state: debate agent:turn 3: all agents running/((agent1:running)|(agent2:idle)|(agent3:idle))
agent state: debate agent:turn 3: all agents running/((agent1:idle)|(agent2:idle)|(agent3:idle))
agent state: debat

In [7]:
from collections.abc import Sequence


n = 0
def reversed_agent_strategy(agents: Sequence[BaseAgent]) -> BaseAgent:
    global n
    n -= 1
    return agents[n % len(agents)]

debate_agent.max_turns = 9
debate_agent.pick_strategy = reversed_agent_strategy

query = "In recent reports, reasearchers have analyzed the plastic pollution in water including lakes, oceans. The study found an average concentration of nanoplastic near coastlines of 25 milligrams per cubic meter of water. Nanoplastics are tiny enough that they can easily infiltrate the bodies of living creatures. For fish and other animals that live in the ocean, that means constant exposure that builds up over time. Propose a solution for better management of plastic waste in the near future."
message = debate_agent.execute(AgentMessage(query=query))

print(f"message len: {len(message.responses)}")
for agent_name, response in message.responses:
    print(f"{agent_name}\n{response}")
    print("-" * 50)
    print()

agent state: debate agent:turn 0: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 0: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:running))
agent state: debate agent:turn 0: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 1: agent agent2 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 1: agent agent2 running/((agent1:idle)-(agent2:running)-(agent3:idle))
agent state: debate agent:turn 1: agent agent2 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 2: agent agent1 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 2: agent agent1 running/((agent1:running)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 2: agent agent1 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
agent state: debate agent:turn 3: agent agent3 running/((agent1:idle)-(agent2:idle)-(agent3:idle))
a